In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/bersonperson/sfdat28/master/data/App%20Weekly%20Review.csv')
df.head()

,ID,Featured,Pct_of_Active_(7d)_Teams_with_App_Installed,Active_(7d)_Installed_Teams,Pct_of_Installed_Teams_Using_App,App_Active_(7d)_Teams,App_Active_(7d)_Teams_Last_Week,Net_Growth,Pct_Growth,New,Pct_New,Resurrected,Retained,Pct_Retention,Churned,Pct_Churn,Stale
0,1,no,25.74,167239,29.50%,49359,45135,4224,9.40%,6839,15.20%,14117,28403,62.90%,16732,37.10%,112999
1,2,no,23.38,151886,43.30%,65718,61338,4380,7.10%,4048,6.60%,13135,48535,79.10%,12803,20.90%,77256
2,3,no,16.31,105937,56.00%,59309,58475,834,1.40%,1998,3.40%,5098,52213,89.30%,6262,10.70%,34088
3,4,no,15.26,99137,14.80%,14681,14445,236,1.60%,447,3.10%,2337,11897,82.40%,2548,17.60%,16424
4,5,no,14.68,95338,14.00%,13384,12731,653,5.10%,1757,13.80%,4091,7536,59.20%,5195,40.80%,30987


In [45]:
df['Featured'] = df.Featured.map({'no':0, 'yes':1})
df.Pct_Growth.mean()
# df.groupby('Featured').Pct_Growth.mean().plot(kind='bar')

TypeError: Could not convert 9.40%7.10%1.40%1.60%5.10%-1.70%4.10%1.90%1.10%16.20%2.60%3.20%3.60%6.70%0.10%6.70%21.80%1.20%0.50%2.10%-1.30%0.50%-1.80%15.00%2.80%1.10%12.00%0.80%7.70%1.50%2.90%7.60%9.30%1.90%3.40%1.20%1.00%25.60%0.10%-0.10%4.80%0.40%12.60%-1.30%2.70%-2.00%-1.50%6.90%0.40%6.50%4.90%7.80%2.30%10.20%2.20%2.80%-1.60%0.10%16.70%-40.90%10.60%4.00%3.90%5.20%-2.80%0.50%-30.80%-2.30%4.20%6.40%12.50%4.20%2.30%11.50%-0.10%-2.70%4.10%11.70%3.20%11.80%0.60%-0.80%20.20%-0.90%18.20%1.80%1.20%1.50%3.60%3.50%3.40%3.90%-1.40%2.50%1.20%-6.60%18.50%5.20%0.70%0.80%1.60%2.40%10.40%5.10%7.80%3.50%6.50%5.40%0.80%0.40%4.20%-4.20%0.30%0.90%-1.00%-16.80%11.60%2.20%8.70%-4.80%-7.00%0.50%-1.20%9.10%0.10%-1.70%-1.80%-3.90%1.80%0.00%0.10%3.20%6.60%1.10%5.60%8.50%6.80%31.40%9.90%3.30%3.70%14.90%-0.80%10.30%2.50%3.90%-0.70%2.70%-0.90%0.90%1.70%6.60%22.00%1.90%43.90%7.30%5.30%9.30%1.30%3.00%-2.70%-0.50%3.50%0.50%4.00%-1.40%0.40%6.50%4.10%0.40%-1.10%-25.40%-2.20%19.50%6.10%3.00%91.30%3.50%43.50%4.40%54.80